In [1]:
%load_ext autoreload
%autoreload 2

import sys
#sys.path.append("../") 


In [12]:
from models.coherence_models import BigramCoherence
from utils.lm_utils import Corpus, SentCorpus
import argparse
import config
from add_args import add_bigram_args
from utils.data_utils import DataSet
from preprocess import get_lm_hidden
import torch
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import DataLoader


In [3]:
embed_dim = 2048
parser = argparse.ArgumentParser()
add_bigram_args(parser)
args = parser.parse_args([])

dataset = DataSet(config.DATASET[args.data_name])
#dataset.data_path = "../" + dataset.data_path

train_dataset = dataset.load_train(args.portion)
test_dataset = dataset.load_test(args.portion)

corpus = Corpus(train_dataset.file_list, test_dataset.file_list, config.WORD_EMBEDDING)
sent_embedding = get_lm_hidden(args.data_name, "lm_" + args.data_name, corpus, limit=100)


    
kwargs = {
    "embed_dim": embed_dim,
    "sent_encoder": sent_embedding,
    "hparams": {
        "loss": args.loss,
        "input_dropout": args.input_dropout,
        "hidden_state": args.hidden_state,
        "hidden_layers": args.hidden_layers,
        "hidden_dropout": args.hidden_dropout,
        "num_epochs": args.num_epochs,
        "margin": args.margin,
        "lr": args.lr,
        "l2_reg_lambda": args.l2_reg_lambda,
        "use_bn": args.use_bn,
        "task": "discrimination",
        "bidirectional": args.bidirectional,
    }
}

(40812, 300)


100%|██████████| 100/100 [00:06<00:00, 15.52it/s]


In [4]:
model = BigramCoherence(**kwargs)

In [36]:
model = torch.load("./checkpoint/wsj_bigram-0.9379.pth", map_location=torch.device('cpu'))
model.use_cuda = False

/Users/daniter/anaconda3/envs/coh/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/daniter/anaconda3/envs/coh/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/daniter/anaconda3/envs/coh/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the 

In [7]:
test_dataset = dataset.load_test(args.portion)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
test_df = dataset.load_test_perm()

In [37]:
test = test_dataloader
df = test_df
debug = True
correct_pred = [0, 0, 0]
total_samples = [0, 0, 0]

if debug:
    all_pos_scores = []
    all_neg_scores = []

model.discriminator.train(False)
i = 0
for article in test:
    i += 1
    if i == 2:
        break
    sentences = df.loc[article[0], "sentences"].split("<PUNC>")
    sent_num = len(sentences)
    sentences = ["<SOA>"] + sentences + ["<EOA>"]
    neg_sentences_list = df.loc[article[0],
                                "neg_list"].split("<BREAK>")
    neg_sentences_list = [s.split("<PUNC>")
                          for s in neg_sentences_list]

    pos_sent1 = sentences[:-1]
    pos_sent1 = model.encode(pos_sent1)
    pos_sent2 = sentences[1:]
    pos_sent2 = model.encode(pos_sent2)
    pos_scores = model.discriminator(pos_sent1, pos_sent2)
    mean_pos_score = pos_scores.mean().data.cpu().numpy()

    if debug:
        all_pos_scores.append(pos_scores.data.cpu().numpy().squeeze())

    mean_neg_scores = []
    for neg_sentences in neg_sentences_list:
        neg_sentences = ["<SOA>"] + neg_sentences + ["<EOA>"]
        neg_sent1 = neg_sentences[:-1]
        neg_sent1 = model.encode(neg_sent1)
        neg_sent2 = neg_sentences[1:]
        neg_sent2 = model.encode(neg_sent2)
        neg_scores = model.discriminator(neg_sent1, neg_sent2)
        mean_neg_score = neg_scores.mean().data.cpu().numpy()
        mean_neg_scores.append(mean_neg_score)

        if debug:
            all_neg_scores.append(
                neg_scores.data.cpu().numpy().squeeze())

    for mean_neg_score in mean_neg_scores:
        if mean_pos_score < mean_neg_score:
            for i in range(3):
                lower_bound = model.intervals[i]
                upper_bound = model.intervals[i + 1]
                if (sent_num > lower_bound) and (sent_num <= upper_bound):
                    correct_pred[i] += 1
        for i in range(3):
            lower_bound = model.intervals[i]
            upper_bound = model.intervals[i + 1]
            if (sent_num > lower_bound) and (sent_num <= upper_bound):
                total_samples[i] += 1
# model.discriminator.train(True)
accs = np.true_divide(correct_pred, total_samples)
acc = np.true_divide(np.sum(correct_pred), np.sum(total_samples))

if debug:
    all_pos_scores = np.concatenate(all_pos_scores)
    all_neg_scores = np.concatenate(all_neg_scores)

    import pandas as pd

    print('pos score stats')
    print(pd.DataFrame(all_pos_scores).describe())

    print('neg score stats')
    print(pd.DataFrame(all_neg_scores).describe())

pos score stats
                  0
count  23476.000000
mean       0.125422
std        3.443641
min      -26.557003
25%       -1.327920
50%        0.366005
75%        1.852593
max       17.844749
neg score stats
                   0
count  469520.000000
mean        1.605783
std         3.498630
min       -26.557003
25%        -0.318012
50%         1.167307
75%         3.043002
max        24.444691


In [38]:
acc

0.9417457305502847

In [19]:
s = df.loc["./data/parsed_wsj/20/wsj_2052", "sentences"].split("<PUNC>")

In [60]:
print(s[0])
print(s1[0])

When the Soviets announced their last soldier had left Afghanistan in February , the voices of skepticism were all but drowned out by an international chorus of euphoria .
President Chen Shui - bian visited the Nicaraguan National Assembly on August 17 , where he received a medal from the president of the assembly , Ivan Escobar Fornos .


In [57]:
#s1 = s[:-1][:1]
#s2 = s[1:][:1]
pos_sent1 = model.encode(s1)
pos_sent2 = model.encode(s2)
model.discriminator(pos_sent1, pos_sent2)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.

In [46]:
pos_sent1

tensor([[ 1.2381e-01,  2.9231e-01,  8.7722e-01,  ...,  2.4608e-01,
         -4.1085e-05,  7.4488e-01]])

In [86]:
list(model.sent_encoder.values())[0]

array([ 6.6557139e-01,  7.4483222e-01,  8.2124192e-01, ...,
        7.8131747e-01, -4.4397300e-05,  8.9844376e-01], dtype=float32)

In [89]:
import os, pickle
from tqdm import tqdm
from torch.autograd import Variable
from models.language_models import LanguageModel

def one_the_fly_encoder(sentences, lm_name="lm_" + args.data_name, corpus=corpus):
    with open(os.path.join(config.CHECKPOINT_PATH, lm_name + "_forward.pkl"), "rb") as f:
        hparams = pickle.load(f)

    kwargs = {
        "vocab_size": corpus.glove_embed.shape[0],
        "embed_dim": corpus.glove_embed.shape[1],
        "corpus": corpus,
        "hparams": hparams,
    }

    forward_lm = LanguageModel(**kwargs)
    forward_lm.load(os.path.join(config.CHECKPOINT_PATH, lm_name + "_forward.pt"))
    forward_lm = forward_lm.lm
    forward_lm.eval()

    backward_lm = LanguageModel(**kwargs)
    backward_lm.load(os.path.join(config.CHECKPOINT_PATH, lm_name + "_backward.pt"))
    backward_lm = backward_lm.lm
    backward_lm.eval()

    embed_dict = {}
    ini_hidden = forward_lm.init_hidden(1)
    for sent in sentences:
        fs = [corpus.vocab[w] for w in ['<eos>'] + sent.split() + ['<eos>']]
        fs = torch.LongTensor(fs).unsqueeze(1)
        if torch.cuda.is_available():
            fs = fs.to('cuda')
        fout = forward_lm.encode(fs, ini_hidden)
        fout = torch.max(fout, 0)[0].squeeze().data.cpu().numpy().astype(np.float32)

        bs = [corpus.vocab[w] for w in ['<eos>'] + sent.split()[::-1] + ['<eos>']]
        bs = torch.LongTensor(bs).unsqueeze(1)
        if torch.cuda.is_available():
            bs = bs.to('cuda')
        bout = backward_lm.encode(bs, ini_hidden)
        bout = torch.max(bout, 0)[0].squeeze().data.cpu().numpy().astype(np.float32)

        embed_dict[sent] = np.hstack((fout, bout))
    np.random.seed(0)
    embed_dict["<SOA>"] = np.random.uniform(size=2048).astype(np.float32)
    embed_dict["<EOA>"] = np.random.uniform(size=2048).astype(np.float32)
    return embed_dict

In [92]:
s1 = ["President Chen Shui - bian visited the Nicaraguan National Assembly on August 17 , where he received a medal from the president of the assembly , Ivan Escobar Fornos ."]
s2 = ["On August 25 President Chen Shui - bian wrapped up his first overseas trip since taking office , swinging through three countries in Latin America and another three in Africa ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
model.discriminator(pos_sent1, pos_sent2)

tensor([-1.6085], grad_fn=<DivBackward0>)

In [93]:
s1 = ["President Chen Shui - bian visited the Nicaraguan National Assembly on August 17 , where he received a medal from the president of the assembly , Ivan Escobar Fornos ."]
s2 = ["On August 25 Ivan Escobar Fornos wrapped up his first overseas trip since taking office , swinging through three countries in Latin America and another three in Africa ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
model.discriminator(pos_sent1, pos_sent2)

tensor([-0.7936], grad_fn=<DivBackward0>)

In [121]:
s1 = ["Chen Shui - bian left on August 13 for a 13 - day trip that included state visits to the Latin American nations of the Dominican Republic , Nicaragua , Costa Rica , as well as the west African nations of Gambia , Burkina Faso , and Chad ."]
s2 = ["Before boarding the plane on the 13th , Chen Shui - bian delivered a short speech in which he stated , \" Taiwan must stand up and be counted . \""]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

s1 = ["Chen Shui - bian left on August 13 for a 13 - day trip that included state visits to the Latin American nations of the Dominican Republic , Nicaragua , Costa Rica , as well as the west African nations of Gambia , Burkina Faso , and Chad ."]
s2 = ["Before boarding the plane on the 13th , Burkina Faso delivered a short speech in which he stated , \" Taiwan must stand up and be counted . \""]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))


tensor([3.1014], grad_fn=<DivBackward0>)
tensor([3.5262], grad_fn=<DivBackward0>)


In [124]:
s1 = ["He attended the presidential inauguration there on the 16th , and was able to meet with the leaders and representatives of friendly nations from around the world ."]
s2 = ["On the 17th he continued on to Nicaragua , where he met with President Arnoldo Aleman and delivered a speech to the Nicaraguan National Assembly ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

s1 = ["He attended the presidential inauguration there on the 16th , and was able to meet with the leaders and representatives of friendly nations from around the world ."]
s2 = ["On the 17th leaders and representatives continued on to Nicaragua , where he met with President Arnoldo Aleman and delivered a speech to the Nicaraguan National Assembly ."]

sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))



tensor([-0.5484], grad_fn=<DivBackward0>)
tensor([-0.3501], grad_fn=<DivBackward0>)


In [126]:

s1 = ["The next day , Chen took advantage of the upcoming presidential inauguration to meet with leaders from a number of countries with which the ROC has diplomatic relations , including the vice-president of El Salvador and the second vice president of Panama ."]
s2 = ["On the 16th Chen attended the inauguration of President Hipolito Mejia ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

s1 = ["The next day , Chen took advantage of the upcoming presidential inauguration to meet with leaders from a number of countries with which the ROC has diplomatic relations , including the vice-president of El Salvador and the second vice president of Panama ."]
s2 = ["On the 16th the second vice president of Panama attended the inauguration of President Hipolito Mejia ."]

sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

tensor([-0.6480], grad_fn=<DivBackward0>)
tensor([-1.1668], grad_fn=<DivBackward0>)


In [130]:

s1 = ["Costa Rica 's President Miguel Rodriguez also accompanied President Chen on a tour of a Taiwanese export processing zone , an orchid farm run by Taiwan Sugar Corporation , and a vocational training school ."]
s2 = ["President Chen departed on the 20th for Africa , where the ROC has eight diplomatic partners ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

s1 = ["Costa Rica 's President Miguel Rodriguez also accompanied President Chen on a tour of a Taiwanese export processing zone , an orchid farm run by Taiwan Sugar Corporation , and a vocational training school ."]
s2 = ["President Rodriguez departed on the 20th for Africa , where the ROC has eight diplomatic partners ."]

sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

tensor([-1.8209], grad_fn=<DivBackward0>)
tensor([-2.2082], grad_fn=<DivBackward0>)


In [131]:

s1 = ["Later , he studied in France for five years , where he became well versed in the various schools of modern Western music ."]
s2 = ["In 1959 he returned to introduce modern Western classical music to his homeland ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

s1 = ["Later , he studied in France for five years , where he became well versed in the various schools of modern Western music ."]
s2 = ["In 1959 France returned to introduce modern Western classical music to its homeland ."]
sents_encoded = one_the_fly_encoder(s1+s2)
pos_sent1 = Variable(torch.from_numpy(np.array(sents_encoded[s1[0]])))
pos_sent2 = Variable(torch.from_numpy(np.array(sents_encoded[s2[0]])))
print(model.discriminator(pos_sent1, pos_sent2))

tensor([0.1598], grad_fn=<DivBackward0>)
tensor([1.1417], grad_fn=<DivBackward0>)
